In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import datetime as dt
import re
import pymysql
import keys


In [3]:

schema = "gans_escooter" # name for schema in database
host = "IP SQl instance"
user = "root"
password = keys.connection_password_
port = 3306

connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

In [4]:
cities_list = ["Berlin", "Hamburg", "Munich", "Cologne"]

In [5]:
def scrape_city_data(cities_list):
 cities_data = []
 API_key = keys.api_keys
 for city in cities_list:
        cities=requests.get(f"https://api.openweathermap.org/data/2.5/forecast?q={city}&cnt={1}&appid={API_key}")
        cities_json = cities.json()
        land= cities_json['city']['country']
        lat= cities_json['city']['coord']['lat']
        lon= cities_json['city']['coord']['lon']
        #pop= cities_json['city']['population']
        city_data= {
                 'city_name': city,
                 'country' : land,
                 'latitude': lat,
                 'longitude': lon,
                  }
         # Add row to list
        cities_data.append(city_data)
  # Combine all rows into data frame
 return pd.DataFrame(cities_data)

In [6]:
cities_static_df = scrape_city_data(cities_list)
cities_static_df

,city_name,country,latitude,longitude
0,Berlin,DE,52.5244,13.4105
1,Hamburg,DE,53.5500,10.0000
2,Munich,DE,48.1374,11.5755
3,Cologne,DE,50.9333,6.9500


In [7]:
cities_static_df.to_sql('cities', 
                  if_exists='append',
                  con=connection_string,
                  index=False)  # send data to sql #read data from sql

OperationalError: (pymysql.err.OperationalError) (2003, "Can't connect to MySQL server on '34.38.32.125' (timed out)")
(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [57]:
cities_from_sql = pd.read_sql("cities", con=connection_string) #read data from sql
cities_from_sql

,city_id,city_name,country,latitude,longitude
0,1,Berlin,DE,52.5244,13.4105
1,2,Hamburg,DE,53.5500,10.0000
2,3,Munich,DE,48.1374,11.5755
3,4,Mumbai,IN,19.0144,72.8479
4,5,Berlin,DE,52.5244,13.4105
5,6,Hamburg,DE,53.5500,10.0000
6,7,Munich,DE,48.1374,11.5755
7,8,Mumbai,IN,19.0144,72.8479
8,9,Berlin,DE,52.5244,13.4105
9,10,Hamburg,DE,53.5500,10.0000


In [58]:
cities_list = ["Berlin", "Hamburg", "Munich", "Cologne"]

In [59]:

cities_data = []
def scrape_city_population_data(cities_list):
    
    
    for city in cities_list:
        url = f"https://en.wikipedia.org/wiki/{city}"
        response = requests.get(url)
        city_soup = BeautifulSoup(response.content, 'html.parser')

    
        population = city_soup.find(string=re.compile("Population")).find_next("td").get_text().replace(',', '')
        pop_int = int(population)

        pop_census_year_full_text = (
            city_soup
            .find(string="Population")
            .find_next(class_='ib-settlement-fn')
            .get_text()
        )
        pop_census_year = re.findall('\((.{4}).*\)', pop_census_year_full_text)[0]
       
       
        cities_data.append({
            "city_name": city,
            "population": pop_int,
            'census_year': pop_census_year,
            "data_collection_timestamp": dt.datetime.now().strftime('%d/%m/%y')
        })

    return pd.DataFrame(cities_data)

# Example usage

citiess_dyanmics_df = scrape_city_population_data(cities_list)
citiess_dyanmics_df

<>:20: SyntaxWarning: invalid escape sequence '\('
<>:20: SyntaxWarning: invalid escape sequence '\('
C:\Users\dtung\AppData\Local\Temp\ipykernel_13508\2236838958.py:20: SyntaxWarning: invalid escape sequence '\('
  pop_census_year = re.findall('\((.{4}).*\)', pop_census_year_full_text)[0]


,city_name,population,census_year,data_collection_timestamp
0,Berlin,3596999,2022,28/01/25
1,Hamburg,1964021,2023,28/01/25
2,Munich,1510378,2023,28/01/25
3,Cologne,1087353,2023,28/01/25


In [60]:
pop_data_df = scrape_city_population_data(cities_from_sql['city_name']) #Extracts the city_name column
pop_data_df

,city_name,population,census_year,data_collection_timestamp
0,Berlin,3596999,2022,28/01/25
1,Hamburg,1964021,2023,28/01/25
2,Munich,1510378,2023,28/01/25
3,Cologne,1087353,2023,28/01/25
4,Berlin,3596999,2022,28/01/25
5,Hamburg,1964021,2023,28/01/25
6,Munich,1510378,2023,28/01/25
7,Mumbai,12442373,2011,28/01/25
8,Berlin,3596999,2022,28/01/25
9,Hamburg,1964021,2023,28/01/25


In [61]:
citiess_dyanmics_df= (
    pop_data_df.merge(cities_from_sql,
                            on = 'city_name',
                                   how= 'inner')
                [['city_id', 'population', 'census_year', 'data_collection_timestamp']]
)
citiess_dyanmics_df                  

,city_id,population,census_year,data_collection_timestamp
0,1,3596999,2022,28/01/25
1,5,3596999,2022,28/01/25
2,9,3596999,2022,28/01/25
3,13,3596999,2022,28/01/25
4,17,3596999,2022,28/01/25
...,...,...,...,...
458,28,1087353,2023,28/01/25
459,32,1087353,2023,28/01/25
460,36,1087353,2023,28/01/25
461,40,1087353,2023,28/01/25


In [62]:
citiess_dyanmics_df .to_sql('populations',
                  if_exists='append',
                  con=connection_string,
                  index=False) # send data to sql 

463

In [63]:
citiess_dyanmics_df  = pd.read_sql("populations", con=connection_string) #read data from sql
citiess_dyanmics_df 

,cities_data_id,city_id,population,census_year,data_collection_timestamp
0,1,1,3596999,2022,2026-01-25
1,2,5,3596999,2022,2026-01-25
2,3,9,3596999,2022,2026-01-25
3,4,13,3596999,2022,2026-01-25
4,5,17,3596999,2022,2026-01-25
...,...,...,...,...,...
1858,1859,28,1087353,2023,2028-01-25
1859,1860,32,1087353,2023,2028-01-25
1860,1861,36,1087353,2023,2028-01-25
1861,1862,40,1087353,2023,2028-01-25


In [64]:
def update_forecast_table(connection_string):
    # Pull city data from database to get corresponding forcasts
    cities_df= pd.read_sql('cities',  con= connection_string)
    forecasts = []
    # Loop through all the cities in the cities_dynamic table pulled from database
    for i, row in cities_df.iterrows(): #iteration through the rows of df
        lat= row['latitude']
        lon= row['longitude']
        #Make API request
        params= {
             'lat': lat,
              'lon': lon,
              'appid': keys.api_keys
        }
        response= requests.get(url= "https://api.openweathermap.org/data/2.5/forecast?",
                              params=params)
        weather_data = response.json()
        # Loop through all the forcast
        for forecast in weather_data['list']:
            #Locate desired information
            forecast_dict= {
                'city_id': row['city_id'],
                'forecast_time': forecast['dt_txt'],
                'temparature': round(forecast['main']['temp'] - 273.15, 2),  # Rounded to 2 decimals
                'feels_like': round(forecast['main']['feels_like'] - 273.15, 2),  # Rounded to 2 decimals
                'wind_speed': round(forecast['wind']['speed'], 2),
                'weather_condition' : forecast['weather'][0]['description'] 
            }
            forecasts.append(forecast_dict)
    forecast_df = pd.DataFrame(forecasts)
    forecast_df
    forecast_df.to_sql('forecasts',
                if_exists='append',
                con=connection_string,
                index=False)      # send data to sql


In [65]:
#Calling scraping function to scrape data
update_forecast_table(connection_string)

In [66]:
forecasts_df  = pd.read_sql("forecasts", con=connection_string) #read data from sql
forecasts_df

,forecast_id,city_id,forecast_time,temparature,feels_like,wind_speed,weather_condition
0,1,1,2025-01-26 12:00:00,8.49,5.49,5.55,scattered clouds
1,2,1,2025-01-26 15:00:00,8.18,5.00,5.82,light rain
2,3,1,2025-01-26 18:00:00,4.06,-0.56,6.65,overcast clouds
3,4,1,2025-01-26 21:00:00,4.03,0.53,4.24,overcast clouds
4,5,1,2025-01-27 00:00:00,3.78,0.78,3.36,overcast clouds
...,...,...,...,...,...,...,...
15515,15516,44,2025-02-02 00:00:00,0.63,0.63,0.94,clear sky
15516,15517,44,2025-02-02 03:00:00,0.15,0.15,1.11,clear sky
15517,15518,44,2025-02-02 06:00:00,-0.15,-0.15,1.11,clear sky
15518,15519,44,2025-02-02 09:00:00,1.52,1.52,0.75,clear sky


In [34]:
airports_data = []
airports_cities_data = []

# Loop through cities in cities table
for i, row in cities_from_sql.iterrows():

    # Construct request
    url = "https://aerodatabox.p.rapidapi.com/airports/search/location"
    params = {"withFlightInfoOnly":"true",
                "lat":row['latitude'],
                'lon':row['longitude'],
                'radiusKm':"50",
                'limit':10}
    headers = {
        "X-RapidAPI-Key": keys.RapidAPI_Key,
        "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
    }
    # Request from API
    response = requests.get(url, headers=headers, params=params)
    airports_json = response.json()

    # Loop through airports in response
    for airport in airports_json['items']:
        # Gather data for database
        airport_data = {
            'iata': airport['iata'],
            'icao': airport['icao'],
            'airport_name': airport['name'],
            'latitude': airport['location']['lat'],
            'longitude': airport['location']['lon']
        }
        # Add to list of airport data
        airports_data.append(airport_data)

        airport_city_data = {
            'iata': airport['iata'],
            'city_id': row['city_id']
        }
        # Add to list of airport-city connections
        airports_cities_data.append(airport_city_data)

# Convert to data frames
airports_df = pd.DataFrame(airports_data)
# Remove any duplicate airports
airports_df.drop_duplicates(inplace=True)
cities_airports_df = pd.DataFrame(airports_cities_data)

In [20]:
airports_df

,iata,icao,airport_name,latitude,longitude
0,BER,EDDB,Berlin Brandenburg,52.35139,13.493889
1,HAM,EDDH,Hamburg,53.63040,9.988229
2,MUC,EDDM,Munich,48.35380,11.786100
3,BOM,VABB,Mumbai Chhatrapati Shivaji,19.08870,72.867900
23,CGN,EDDK,Cologne Bonn,50.86590,7.142739
24,DUS,EDDL,Düsseldorf,51.28950,6.766779


In [21]:
cities_airports_df

,iata,city_id
0,BER,1
1,HAM,2
2,MUC,3
3,BOM,4
4,BER,5
5,HAM,6
6,MUC,7
7,BOM,8
8,BER,9
9,HAM,10


In [ ]:
airports_df.to_sql(
    'airports',
    if_exists='append',
    con=connection_string,
    index=False)
cities_airports_df.to_sql(
    'cities_airports',
    if_exists='append',
    con=connection_string,
    index=False)      # send data to sql

45

In [34]:
airports_df  = pd.read_sql("airports", con=connection_string) # read data from sql
airports_df 

,iata,icao,airport_name,longitude,latitude
0,BER,EDDB,Berlin Brandenburg,13.49390,52.3514
1,BOM,VABB,Mumbai Chhatrapati Shivaji,72.86790,19.0887
2,CGN,EDDK,Cologne Bonn,7.14274,50.8659
3,DUS,EDDL,Düsseldorf,6.76678,51.2895
4,HAM,EDDH,Hamburg,9.98823,53.6304
5,MUC,EDDM,Munich,11.78610,48.3538


In [38]:
cities_airports_df  = pd.read_sql("cities_airports", con=connection_string) # read data from sql
cities_airports_df

,city_id,iata
0,1,BER
1,4,BOM
2,2,HAM
3,3,MUC


In [89]:
def request_flights_data():
    # Get connection string
    #connection_string = connect_to_gans_local()  # Ensure this function is defined elsewhere
    connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'
    # Pull airports from the database to get corresponding flights
    airports_df = pd.read_sql('airports', con=connection_string)
    flights_data = []
    # Create time windows for 12hr searches
    tomorrow = dt.datetime.now() + dt.timedelta(days=1)
    tomorrow_date = tomorrow.strftime('%Y-%m-%d')
    # Time windows for morning and afternoon
    morning_start = f'{tomorrow_date}T00:00'
    morning_end = f'{tomorrow_date}T11:59'
    afternoon_start = f'{tomorrow_date}T12:00'
    afternoon_end = f'{tomorrow_date}T23:59'
    day_parts = [(morning_start, morning_end), (afternoon_start, afternoon_end)]
    # Loop through airports
    for i, row in airports_df.iterrows():
        iata_code = row['iata']  # Using IATA code from the row
        # Loop for both halves of the day
        for time_start, time_end in day_parts:
            # Construct request URL
            base_url = "https://aerodatabox.p.rapidapi.com/flights/airports"
            path_params = f"/iata/{iata_code}/{time_start}/{time_end}"
            full_url = base_url + path_params
            querystring = {
                "withLeg": "true",
                "direction": "Arrival",
                "withCancelled": "False",
                "withCodeshared": "False",
                "withCargo": "False",
                "withPrivate": "False",
                "withLocation": "false"
            }
            headers = {
                "x-rapidapi-key": keys.RapidAPI_Key,
                "x-rapidapi-host": "aerodatabox.p.rapidapi.com"
            }
            # Request data from API
            response = requests.get(full_url, headers=headers, params=querystring)
            if response.status_code == 200:
                flight_json = response.json()
                # Check if 'arrivals' key exists in the response
                if 'arrivals' in flight_json:
                    # Loop through flight data
                    for flight in flight_json['arrivals']:
                        scheduled_arrival = flight['arrival']['scheduledTime']['local']
                        updated_arrival = (
                            flight['arrival'].get('revisedTime', {'local': scheduled_arrival})['local']
                        )
                        # Arrange flight data into a dictionary
                        flight_data = {
                            'flight_number': flight['number'],
                            'iata': iata_code,
                            'scheduled_arrival_time': scheduled_arrival[:-6],  # Remove timezone part
                            'updated_arrival_time': updated_arrival[:-6],  # Remove timezone part
                            'departure_location': flight['departure']['airport']['name']
                        }
                        flights_data.append(flight_data)
                else:
                    print(f"No arrivals found for {iata_code} between {time_start} and {time_end}")
            else:
                print(f"Error fetching data for {iata_code} between {time_start} and {time_end}: {response.status_code}")
    # Convert list of flight data to a DataFrame
    flights_df = pd.DataFrame(flights_data)
    # Send data to database
    flights_df.to_sql('flights',
                      if_exists='append',
                      con=connection_string,
                      index=False)  # send data to sql

In [90]:
request_flights_data()

In [ ]:
flights_df  = pd.read_sql("flights", con=connection_string) #read data from sql
flights_df